In [254]:
# !pip install catboost
# !pip install fancyimpute
# !wget https://he-s3.s3.ap-southeast-1.amazonaws.com/media/hackathon/amexpert-code-lab/credit-card-default-risk-5-95cbc85f/892bcbe6493411ec.zip
# !unzip /content/892bcbe6493411ec.zip

In [255]:
import pandas as pd
import numpy as np
import seaborn as sns

In [256]:
train=pd.read_csv("/content/dataset/train.csv")
train.head()

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,CST_115179,ita Bose,46,F,N,Y,0.0,107934.04,612.0,Unknown,1.0,1.0,33070.28,18690.93,73,544.0,2,1,1
1,CST_121920,Alper Jonathan,29,M,N,Y,0.0,109862.62,2771.0,Laborers,2.0,0.0,15329.53,37745.19,52,857.0,0,0,0
2,CST_109330,Umesh Desai,37,M,N,Y,0.0,230153.17,204.0,Laborers,2.0,0.0,48416.60,41598.36,43,650.0,0,0,0
3,CST_128288,Rie,39,F,N,Y,0.0,122325.82,11941.0,Core staff,2.0,0.0,22574.36,32627.76,20,754.0,0,0,0
4,CST_151355,McCool,46,M,Y,Y,0.0,387286.00,1459.0,Core staff,1.0,0.0,38282.95,52950.64,75,927.0,0,0,0


In [257]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45528 entries, 0 to 45527
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              45528 non-null  object 
 1   name                     45528 non-null  object 
 2   age                      45528 non-null  int64  
 3   gender                   45528 non-null  object 
 4   owns_car                 44981 non-null  object 
 5   owns_house               45528 non-null  object 
 6   no_of_children           44754 non-null  float64
 7   net_yearly_income        45528 non-null  float64
 8   no_of_days_employed      45065 non-null  float64
 9   occupation_type          45528 non-null  object 
 10  total_family_members     45445 non-null  float64
 11  migrant_worker           45441 non-null  float64
 12  yearly_debt_payments     45433 non-null  float64
 13  credit_limit             45528 non-null  float64
 14  credit_limit_used(%)  

In [258]:
train=train.dropna()

In [259]:
def encode(data):
  gang=['age','gender','owns_car','owns_house','no_of_children','occupation_type','total_family_members','migrant_worker','credit_limit_used(%)','prev_defaults','default_in_last_6months']
  # gang=['gender','owns_car','owns_house','no_of_children','occupation_type','total_family_members','migrant_worker','prev_defaults','default_in_last_6months']
  cat_data=pd.DataFrame()
  for i in gang:
    status=pd.get_dummies(data[i],prefix=i,drop_first=True)
    for j in status:
      status[j] = status[j].astype(int)
    status.loc[data[i].isnull(),status.columns.str.startswith(i+"_")] = np.nan
    # for j in status.columns:
    #   status[j] = status[j].fillna(status[j].mode()[0])
    cat_data=pd.concat([cat_data,status],axis=1)
  data=pd.concat([data,cat_data],axis=1)
  data.drop(columns=gang,axis=1,inplace=True)
  return data

In [260]:
train=encode(train)

In [261]:
test=pd.read_csv("/content/dataset/test.csv")
test.head()

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months
0,CST_142525,Siva,52,F,Y,N,0.0,232640.53,998.0,Unknown,2.0,0.0,14406.73,26524.40,4,779.0,0,0
1,CST_129215,Scott,48,F,N,N,1.0,284396.79,1338.0,Unknown,3.0,0.0,57479.99,68998.72,70,806.0,0,0
2,CST_138443,Victoria,50,F,N,N,1.0,149419.28,1210.0,Unknown,3.0,0.0,21611.01,25187.80,71,528.0,2,0
3,CST_123812,John McCrank,30,F,N,N,1.0,160437.54,503.0,Laborers,2.0,1.0,28990.76,29179.39,9,815.0,0,0
4,CST_144450,Martinne,52,M,N,Y,0.0,233480.37,157.0,Laborers,2.0,1.0,54213.72,82331.82,82,613.0,1,1


In [262]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11383 entries, 0 to 11382
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              11383 non-null  object 
 1   name                     11383 non-null  object 
 2   age                      11383 non-null  int64  
 3   gender                   11383 non-null  object 
 4   owns_car                 11251 non-null  object 
 5   owns_house               11383 non-null  object 
 6   no_of_children           11193 non-null  float64
 7   net_yearly_income        11383 non-null  float64
 8   no_of_days_employed      11278 non-null  float64
 9   occupation_type          11383 non-null  object 
 10  total_family_members     11352 non-null  float64
 11  migrant_worker           11357 non-null  float64
 12  yearly_debt_payments     11361 non-null  float64
 13  credit_limit             11383 non-null  float64
 14  credit_limit_used(%)  

In [263]:
test=encode(test)

In [264]:
test['customer_id'].dtype

dtype('O')

In [265]:
for i in test.columns:
  if str(test[i].dtype)=='float64':
    test[i].fillna(test[i].mean(),inplace=True)
  elif str(test[i].dtype)=='int64':
    test[i].fillna(test[i].mode(),inplace=True)
  elif str(test[i].dtype)=='category':
    test[i].fillna(test[i].mode(),inplace=True)
  else:
    test[i].fillna(test[i].mode(),inplace=True)

In [266]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11383 entries, 0 to 11382
Columns: 179 entries, customer_id to default_in_last_6months_1
dtypes: float64(177), object(2)
memory usage: 15.5+ MB


In [267]:
sample=pd.read_csv("/content/dataset/sample_submission.csv")
sample.head()

,customer_id,credit_card_default
0,CST_115179,1
1,CST_121920,0
2,CST_109330,0
3,CST_128288,0
4,CST_151355,0


In [268]:
temp=[]
for i in train.columns:
  for j in test.columns:
    if(i==j):
      temp.append(i)

temp.remove('customer_id')
temp.remove('name')
x=train[temp]
y=train['credit_card_default']
z=test[temp]

In [272]:
from xgboost import XGBClassifier
cat = XGBClassifier(max_depth=10)
cat.fit(x,y)
y_pred = cat.predict(z)
final=pd.DataFrame()
final['customer_id']=test['customer_id']
final['credit_card_default']=y_pred
final.to_csv("Final_Credit.csv",index=False)